In [1]:
from fastai.vision.all import *

In [2]:
hand_path = '../../../data/score_hands/'

In [3]:
files = get_files(hand_path)
files[:3]

(#3) [Path('../../../data/score_hands/IMG_1092_swing_1_score_1.npy'),Path('../../../data/score_hands/IMG_0861_swing_0_score_3.npy'),Path('../../../data/score_hands/IMG_1015_swing_2_score_3.npy')]

In [4]:
files[0].name.split('_')[-1].split('.')[0]

'1'

In [5]:
def label_func(fpath): 
    return fpath.name.split('_')[-1].split('.')[0]

In [6]:
label_func(files[0])

'1'

In [7]:
total_pool = np.arange(len(files))
train_idxs = np.random.choice(total_pool, 120, replace=False)
valid_idxs = np.setdiff1d(total_pool, train_idxs)
len(train_idxs), len(valid_idxs)

(120, 36)

In [8]:
train_files = files[train_idxs]
valid_files = files[valid_idxs]

In [9]:
from torchvision import transforms

transform_pipeline = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), # Randomly flip horizontally with 50% probability
    transforms.RandomRotation(degrees=30), # Randomly rotate by up to 30 degrees
    transforms.Resize((224, 224)), # Resize to a common size
    transforms.ToTensor(), # Convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize pixel values
])

In [10]:
from torch.utils.data import Dataset

def label_func(fpath): 
    return fpath.name.split('_')[-1].split('.')[0]

class HandDataset(Dataset):
    def __init__(self, 
                 file_paths,
                 transform=None,
                 idx=None,
                ):
        self.file_paths = file_paths
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        npy_path = self.file_paths[idx]
        npy_hands = np.load(npy_path).astype(np.float32)
        rand_idx = np.random.randint(len(npy_hands))
        npy_hand = npy_hands[rand_idx].transpose(2,0,1)
        #npy_hand = npy_hands[0].transpose(2,0,1)
        score = int(label_func(npy_path)) - 1
        if score == 98:
            print(npy_path)
        return npy_hand, score

In [11]:
train_ds = HandDataset(train_files, transform=transform_pipeline)
valid_ds = HandDataset(valid_files)
len(train_ds), len(valid_ds)

(120, 36)

In [12]:
#train_ds[0]

In [13]:
# from eagle_swing.video_utils import *

In [14]:
# vids = [x[0] for x in train_ds]
# scores = [x[1] for x in train_ds]

In [15]:
# idxs = [12, 15]
# print(scores[idxs[0]: idxs[1]])
# view_videos(vids[idxs[0]:idxs[1]])

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_dataloader = DataLoader(valid_ds, batch_size=8, shuffle=True)
dls = DataLoaders(train_dataloader, test_dataloader)

In [17]:
learn = Learner(dls, xresnet50(n_out=5), 
                metrics=accuracy,
               #loss_func=nn.CrossEntropyLoss()
                loss_func=nn.MSELoss()
               )
learn.fine_tune(base_lr=0.1, epochs=100)

epoch,train_loss,valid_loss,accuracy,time


/home/azaidi/anaconda3/envs/fastdl/lib/python3.10/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (5) must match the size of tensor b (16) at non-singleton dimension 1